<a href="https://colab.research.google.com/github/MarMarhoun/freelance_work/blob/main/side_projects/NLP_projs/eda_streamlit/Image_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project description

This code will provide you with an example of how to use Streamlit, TensorFlow, and an online medical image dataset to create a deep learning model for medical image detection. I will use the "MedNIST" dataset, a large-scale dataset of labeled clinical images, for this example.

First, let's start by installing the necessary libraries:

In [ ]:
!pip install streamlit tensorflow numpy Pillow matplotlib

Next, create a Python script (e.g., app.py) with the following code:



In [ ]:
import streamlit as st
import tensorflow as tf
import numpy as np
import Pillow as pil
from Pillow.Image import Image
import matplotlib.pyplot as plt

# Load the dataset
def load_data(batch_size):
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.medmnist.load_data(test_split=0.2)
    train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(buffer_size=10000).batch(batch_size)
    test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(batch_size)
    return train_dataset, test_dataset

# Model architecture
def create_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Streamlit app
def main():
    st.title("MedNIST: Medical Image Detection")

    batch_size = st.sidebar.slider("Batch Size", 1, 128, 32)
    epochs = st.sidebar.slider("Epochs", 1, 50, 10)

    train_dataset, test_dataset = load_data(batch_size)
    model = create_model()

    if st.sidebar.button("Train Model"):
        model.fit(train_dataset, epochs=epochs)

    if st.sidebar.button("Evaluate Model"):
        model.evaluate(test_dataset)

    if st.sidebar.button("Predict"):
        image_file = st.sidebar.file_uploader("Upload an Image", type=["jpg", "jpeg", "png"])
        if image_file:
            image = Image.open(image_file)
            image = image.resize((28, 28))
            image = np.expand_dims(image, axis=0)
            image = np.expand_dims(image, axis=3)
            image = image / 255.0
            prediction = model.predict(image)
            class_index = np.argmax(prediction)
            class_name = tf.keras.datasets.medmnist.load_data(test_split=0.2)[1][class_index]
            st.write(f"Prediction: {class_name}")
            plt.imshow(image[0], cmap="gray")
            st.pyplot()

if __name__ == "__main__":
    main()

Now, you can run the Streamlit app using the following command:



In [ ]:
streamlit run app.py

This app will allow you to train a deep learning model for medical image detection using the MedNIST dataset. You can also upload your own medical images to test the model's predictions.

## Enhancement

To enhance and add advanced features to the medical images detection code using an online dataset, Streamlit, and TensorFlow, you can consider the following steps:

Data Preprocessing:

Image Normalization: Normalize the pixel values of the medical images to a range between 0 and 1.

Data Augmentation: Apply data augmentation techniques such as rotation, scaling, flipping, and brightness adjustment to increase the size of the training dataset and prevent overfitting.
Model Architecture:

Transfer Learning: Use a pre-trained model such as ResNet, VGG, or DenseNet as a starting point and fine-tune it for the medical image detection task.

Ensemble Learning: Combine multiple models to improve the accuracy and robustness of the medical image detection system.
Model Evaluation:

Cross-Validation: Use k-fold cross-validation to evaluate the performance of the model and reduce the impact of randomness.

Metrics: Use appropriate evaluation metrics such as accuracy, precision, recall, F1-score, and AUC-ROC to assess the performance of the model.
Streamlit App:

User Interface: Create a user-friendly interface using Streamlit to allow users to upload medical images and visualize the detection results.

Model Serving: Serve the trained model using TensorFlow Serving or a similar tool to enable real-time medical image detection.

Here's an example code snippet that demonstrates how to implement some of these advanced features:

In [ ]:
import streamlit as st
import tensorflow as tf
import numpy as np
import Pillow as pil
from Pillow.Image import Image
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Load the dataset
def load_data(batch_size):
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.medmnist.load_data(test_split=0.2)
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=20,
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0.1,
        zoom_range=0.1,
        horizontal_flip=True,
        vertical_flip=True
    )
    test_datagen = ImageDataGenerator(rescale=1./255)
    train_generator = train_datagen.flow(x_train, y_train, batch_size=batch_size)
    test_generator = test_datagen.flow(x_test, y_test, batch_size=batch_size)
    return train_generator, test_generator

# Model architecture
def create_model():
    base_model = tf.keras.applications.ResNet50(weights='imagenet', include_top=False, input_shape=(28, 28, 1))
    x = base_model.output
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dense(1024, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    x = tf.keras.layers.Dense(10, activation='softmax')(x)
    model = tf.keras.Model(inputs=base_model.input, outputs=x)
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Streamlit app
def main():
    st.title("MedNIST: Medical Image Detection")

    batch_size = st.sidebar.slider("Batch Size", 1, 128, 32)
    epochs = st.sidebar.slider("Epochs", 1, 50, 10)

    train_generator, test_generator = load_data(batch_size)
    model = create_model()

    if st.sidebar.button("Train Model"):
        model.fit(train_generator, epochs=epochs, validation_data=test_generator)

    if st.sidebar.button("Evaluate Model"):
        loss, accuracy = model.evaluate(test_generator)
        st.write(f"Loss: {loss}, Accuracy: {accuracy}")

    if st.sidebar.button("Predict"):
        image_file = st.sidebar.file_uploader("Upload an Image", type=["jpg", "jpeg", "png"])
        if image_file:
            image = Image.open(image_file)
            image = image.resize((28, 28))
            image = np.expand_dims(image, axis=0)
            image = np.expand_dims(image, axis=3)
            image = image / 255.0
            prediction = model.predict(image)
            class_index = np.argmax(prediction)
            class_name = tf.keras.datasets.medmnist.load_data(test_split=0.2)[1][class_index]
            st.write(f"Prediction: {class_name}")
            plt.imshow(image[0], cmap="gray")
            st.pyplot()

if __name__ == "__main__":
    main()

This code snippet demonstrates how to use data augmentation, transfer learning, and Streamlit to create a medical image detection system. You can further enhance this code by adding more advanced features such as ensemble learning, cross-validation, and model serving.



## Enhancement 2

To add more enhanced features to the code for medical images detection using an online dataset, Streamlit, and TensorFlow, you can consider the following suggestions:

Data Augmentation: You can use data augmentation techniques such as rotation, scaling, flipping, and brightness adjustment to increase the size of the training dataset and prevent overfitting. This can be done using the ImageDataGenerator class from Keras.

Transfer Learning: You can use a pre-trained model such as ResNet, VGG, or DenseNet as a starting point and fine-tune it for the medical image detection task. This can help in improving the accuracy and reducing the training time.

Ensemble Learning: You can combine multiple models to improve the accuracy and robustness of the medical image detection system. This can be done by training multiple models on different subsets of the training data and combining their predictions using a voting or averaging scheme.

Model Evaluation: You can use cross-validation to evaluate the performance of the model and reduce the impact of randomness. You can also use appropriate evaluation metrics such as accuracy, precision, recall, F1-score, and AUC-ROC to assess the performance of the model.

Streamlit App: You can create a user-friendly interface using Streamlit to allow users to upload medical images and visualize the detection results. You can also serve the trained model using TensorFlow Serving or a similar tool to enable real-time medical image detection.

Here is an example code snippet that demonstrates how to implement some of these advanced features:

In [ ]:
import streamlit as st
import tensorflow as tf
import numpy as np
import Pillow as pil
from Pillow.Image import Image
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Load the dataset
def load_data(batch_size):
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.medmnist.load_data(test_split=0.2)
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=20,
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0.1,
        zoom_range=0.1,
        horizontal_flip=True,
        vertical_flip=True
    )
    test_datagen = ImageDataGenerator(rescale=1./255)
    train_generator = train_datagen.flow(x_train, y_train, batch_size=batch_size)
    test_generator = test_datagen.flow(x_test, y_test, batch_size=batch_size)
    return train_generator, test_generator

# Model architecture
def create_model():
    base_model = tf.keras.applications.ResNet50(weights='imagenet', include_top=False, input_shape=(28, 28, 1))
    x = base_model.output
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dense(1024, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    x = tf.keras.layers.Dense(10, activation='softmax')(x)
    model = tf.keras.Model(inputs=base_model.input, outputs=x)
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Streamlit app
def main():
    st.title("MedNIST: Medical Image Detection")

    batch_size = st.sidebar.slider("Batch Size", 1, 128, 32)
    epochs = st.sidebar.slider("Epochs", 1, 50, 10)

    train_generator, test_generator = load_data(batch_size)
    model = create_model()

    if st.sidebar.button("Train Model"):
        model.fit(train_generator, epochs=epochs, validation_data=test_generator)

    if st.sidebar.button("Evaluate Model"):
        loss, accuracy = model.evaluate(test_generator)
        st.write(f"Loss: {loss}, Accuracy: {accuracy}")

    if st.sidebar.button("Predict"):
        image_file = st.sidebar.file_uploader("Upload an Image", type=["jpg", "jpeg", "png"])
        if image_file:
            image = Image.open(image_file)
            image = image.resize((28, 28))
            image = np.expand_dims(image, axis=0)
            image = np.expand_dims(image, axis=3)
            image = image / 255.0
            prediction = model.predict(image)
            class_index = np.argmax(prediction)
            class_name = tf.keras.datasets.medmnist.load_data(test_split=0.2)[1][class_index]
            st.write(f"Prediction: {class_name}")
            plt.imshow(image[0], cmap="gray")
            st.pyplot()

if __name__ == "__main__":
    main()

This code snippet demonstrates how to use data augmentation, transfer learning, and Streamlit to create a medical image detection system. You can further enhance this code by adding more advanced features such as ensemble learning, cross-validation, and model serving.

